In [85]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.pipeline import Pipeline
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np

In [230]:
# Adds columns for each text variable that will be analyzed. Splits data into overall training and testing sets.
def preprocess(data_set):
    data_set["likely_title_fraud"] = np.nan
    data_set["likely_profile_fraud"] = np.nan
    data_set["likely_desc_fraud"] = np.nan
    data_set["likely_req_fraud"] = np.nan
    data_set["likely_ben_fraud"] = np.nan
    ben = data_set["benefits"].fillna("")
    req = data_set["requirements"].fillna("")
    desc = data_set["description"].fillna("")
    cp = data_set["company_profile"].fillna("")
    tit = data_set["title"].fillna("")
    data_set = data_set.assign(benefits = ben)
    data_set = data_set.assign(requirements = req)
    data_set = data_set.assign(description = desc)
    data_set = data_set.assign(company_profile = cp)
    data_set = data_set.assign(title = tit)
    big_train, big_test, y_train, y_test = train_test_split(data_set, data_set["fraudulent"], test_size=0.2, random_state=5)
    return big_train, big_test

In [31]:
# Removes NA entries, includes all fraudulent cases, samples from non-fraudulent cases, and splits into trianing and
# testing split. X contains feautres of interest as array of the text. Y contains binary fraudulent classification.
def preprocessNLPData(var, data_set):
    data_set = data_set[data_set[var].notna()]
    X_train, X_test, y_train, y_test = train_test_split(data_set[var], data_set["fraudulent"], test_size=0.33, random_state=2)
    X_train = np.array(X_train)
    X_test = np.array(X_test)
    y_train = np.array(y_train)
    y_test = np.array(y_test)
    return X_train, X_test, y_train, y_test

In [32]:
# Creates NLP classifier for specific text variable from data. 
# Splits data into training and testing sets.
# Returns the classifier fitted on training data, along with testing X and y data.
def createNLPClassifier(var, data_set):
    X_train, X_test, y_train, y_test = preprocessNLPData(var, data_set)

    clf = Pipeline([('vect', CountVectorizer()), 
                     ('tfidf', TfidfTransformer()),
                     ('clf', BernoulliNB())])

    clf = clf.fit(X_train, y_train)
    return clf, X_test, y_test

In [33]:
# Uses classifier to predict on testign data.
# Returns overall accuracy of classifer along with confusion matrix of results.
def assessClassifier(clf, X_test, y_test):
    # Predicts class using variable classifier.
    predicted_fraud = clf.predict(X_test)

    # Predicts raw percentages, each observation as an array with two entries: first is prob = 0, second is prob = 1. 
    predicted_perc = clf.predict_proba(X_test)

    accuracy = np.mean(predicted_fraud == y_test)
    c_matrix = confusion_matrix(y_test, predicted_fraud)
    return accuracy, c_matrix

In [34]:
# Makes and assess NLP models for each text variable in the dataset. 
def makeNLPModels(data_set):
    variables = ["description", "requirements", "benefits", "company_profile", "title"]
    accuracies = [0] * 5
    c_matrices = [0] * 5
    for i in range(5):
        clf, X_test, y_test = createNLPClassifier(variables[i], big_train)
        accuracy, confusion_matrix = assessClassifier(clf, X_test, y_test)
        accuracies[i] = accuracy
        c_matrices[i] = confusion_matrix
    return accuracies, c_matrices

In [194]:
# Return: Percentage liklihood that observation is of the fraudulent class.
# Params: Classifier for the specific feature. 
#         X string text variable you want to predict on.
def getFraudulentLiklihood(clf, X):
    if type(X) is not str:
        return np.nan
    else:
        return clf.predict_proba([X])[0][1]


In [35]:
# Uses all of training data to create classifier for the text of the test data.
def makeTestClassifier(train, var):
    X_train = np.array(train[var])
    y_train = np.array(train["fraudulent"])
    
    clf = Pipeline([('vect', CountVectorizer()), 
                     ('tfidf', TfidfTransformer()),
                     ('clf', BernoulliNB())])

    clf = clf.fit(X_train, y_train)
    return clf 

In [188]:
# Get percent liklihood that each test observation text for each variable is fraudulent, based on training classifier.
def getTestLiklihoods(train, test):
    variables = ["description", "requirements", "benefits", "company_profile", "title"]
    liklihoods = ["likely_desc_fraud","likely_req_fraud","likely_ben_fraud","likely_profile_fraud","likely_title_fraud"]
    result = test
    for i in range(5):
        var_train = train[train[variables[i]].notna()]
        clf = makeTestClassifier(var_train, variables[i])
        X_test = np.array(test[variables[i]])
        percentages = np.array(test[liklihoods[i]])
        for j in range(len(percentages)):
            percentages[j] = getFraudulentLiklihood(clf, X_test[j])
        result[liklihoods[i]] =  percentages
    return result

In [239]:
def getTrainLiklihoods(data):  
    leave_out = data.iloc[0:5]
    train = data.drop(data.index[0:5])
    result = getTestLiklihoods(train, leave_out)
    for i in range(5,data.shape[0], 5):
        leave_out = data.iloc[i:i+5]
        train = data.drop(data.index[i:i+5])
        result = result.append(getTestLiklihoods(train, leave_out))
        
    return result
    

In [232]:
# Might need to change this line dependinig on where your file is located. 
origin_data_set = pd.read_csv("cleaned_fake_job_postings.csv")
big_train, big_test = preprocess(origin_data_set)


,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,...,description_length,requirements_length,benefits_length,country,city,likely_title_fraud,likely_profile_fraud,likely_desc_fraud,likely_req_fraud,likely_ben_fraud
1394,16904,customer care representative,"US, OH, Cincinnati",NaN,NaN,lei home enhancements is an ohio based company...,customer care representative i • successfu...,qualificationsrequires a hs diploma or gedup t...,competitive salarybase with commissionsfull be...,0,...,853,233,102,US,other,NaN,NaN,NaN,NaN,NaN
3301,17718,csr,"US, ,",Customer service,NaN,gary cartwright established cartwright propert...,we are seeking customer service reps for our c...,,,0,...,52,0,0,US,other,NaN,NaN,NaN,NaN,NaN
2114,4159,customer service representative,"US, GA, Athens",NaN,NaN,url25e85371e7108e279a4b25d6e965889a44917879b7b...,we are looking for a new addition to our world...,a strong attention to detaila passion for help...,,0,...,1338,160,0,US,athens,NaN,NaN,NaN,NaN,NaN
586,9063,javascript developer,"US, CA, San Carlos",NaN,NaN,for over 15 years casenex has been a leader in...,the datacation division of casenex is looking ...,strong javascript skills in a variety of conte...,full benefits including medical dental life in...,0,...,489,278,276,US,other,NaN,NaN,NaN,NaN,NaN
3343,2397,senior qa engineer,"IN, , Hyderabad",Information Technology,NaN,aptitude staffing solutions has redesigned the...,responsibilities acquire and maintain a compre...,required skills and experienceminimum of five ...,what is offeredour clients core values drive t...,0,...,899,406,2117,IN,other,NaN,NaN,NaN,NaN,NaN
2663,7556,internship copywriting social media at a new ...,"GB, MAN,",NaN,NaN,,location central manchesterrole unpaid interns...,,,0,...,1238,0,0,GB,other,NaN,NaN,NaN,NaN,NaN
651,381,ext js developer,"US, NY, New York",NaN,90000-120000,,the ext js developer is a key member of the de...,required skillsbachelor’s degree in related fi...,full benefits and bonus,0,...,635,543,23,US,new york,NaN,NaN,NaN,NaN,NaN
3743,17682,manufacturing engineer,"US, ,",NaN,NaN,,engineering 100 employee owned windings incurl...,qualified applicants should have a bs manufact...,,0,...,695,185,0,US,other,NaN,NaN,NaN,NaN,NaN
412,366,english teacher abroad,"US, NC, Elon",NaN,NaN,we help teachers get safe amp secure jobs abroad,play with kids get paid for it love travel job...,university degree required tefl tesol celta ...,see job description,0,...,361,126,19,US,other,NaN,NaN,NaN,NaN,NaN
3211,17679,vacancy in halliburton,"GB, ABE, Scotland",NaN,80000-120000,,the human resource department of halliburton o...,interested persons should possess a minimum o...,,0,...,1084,302,0,GB,other,NaN,NaN,NaN,NaN,NaN


In [233]:

# Makes a NB model for each text variable in the training data and assess its accuracy.
accuracies, c_matrices = makeNLPModels(big_train)
for i in range(5):
    print(accuracies[i])
    print(c_matrices[i])

0.8645833333333334
[[831   4]
 [139  82]]
0.8200757575757576
[[834   1]
 [189  32]]
0.8664772727272727
[[834   1]
 [140  81]]
0.7708333333333334
[[609 226]
 [ 16 205]]
0.8409090909090909
[[804  31]
 [137  84]]


In [240]:
# Gets test likelihoods from training data
test = getTestLiklihoods(big_train, big_test)
test

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,...,description_length,requirements_length,benefits_length,country,city,likely_title_fraud,likely_profile_fraud,likely_desc_fraud,likely_req_fraud,likely_ben_fraud
1394,16904,customer care representative,"US, OH, Cincinnati",NaN,NaN,lei home enhancements is an ohio based company...,customer care representative i • successfu...,qualificationsrequires a hs diploma or gedup t...,competitive salarybase with commissionsfull be...,0,...,853,233,102,US,other,4.891793e-02,6.952920e-85,4.077812e-07,5.757123e-07,1.241910e-03
3301,17718,csr,"US, ,",Customer service,NaN,gary cartwright established cartwright propert...,we are seeking customer service reps for our c...,,,0,...,52,0,0,US,other,7.029761e-02,1.000000e+00,2.663880e-13,2.087666e-04,3.139275e-04
2114,4159,customer service representative,"US, GA, Athens",NaN,NaN,url25e85371e7108e279a4b25d6e965889a44917879b7b...,we are looking for a new addition to our world...,a strong attention to detaila passion for help...,,0,...,1338,160,0,US,athens,1.040433e-01,2.234141e-15,2.776337e-22,2.923455e-06,3.139275e-04
586,9063,javascript developer,"US, CA, San Carlos",NaN,NaN,for over 15 years casenex has been a leader in...,the datacation division of casenex is looking ...,strong javascript skills in a variety of conte...,full benefits including medical dental life in...,0,...,489,278,276,US,other,1.352304e-03,1.972185e-17,3.773631e-25,6.175314e-10,2.335342e-03
3343,2397,senior qa engineer,"IN, , Hyderabad",Information Technology,NaN,aptitude staffing solutions has redesigned the...,responsibilities acquire and maintain a compre...,required skills and experienceminimum of five ...,what is offeredour clients core values drive t...,0,...,899,406,2117,IN,other,1.751613e-02,1.000000e+00,7.341322e-10,4.062025e-09,1.000000e+00
2663,7556,internship copywriting social media at a new ...,"GB, MAN,",NaN,NaN,,location central manchesterrole unpaid interns...,,,0,...,1238,0,0,GB,other,3.776838e-05,1.000000e+00,6.669042e-32,2.087666e-04,3.139275e-04
651,381,ext js developer,"US, NY, New York",NaN,90000-120000,,the ext js developer is a key member of the de...,required skillsbachelor’s degree in related fi...,full benefits and bonus,0,...,635,543,23,US,new york,3.103177e-03,1.000000e+00,2.747512e-20,9.800072e-21,1.841299e-03
3743,17682,manufacturing engineer,"US, ,",NaN,NaN,,engineering 100 employee owned windings incurl...,qualified applicants should have a bs manufact...,,0,...,695,185,0,US,other,2.054001e-02,1.000000e+00,7.791032e-01,2.528335e-04,3.139275e-04
412,366,english teacher abroad,"US, NC, Elon",NaN,NaN,we help teachers get safe amp secure jobs abroad,play with kids get paid for it love travel job...,university degree required tefl tesol celta ...,see job description,0,...,361,126,19,US,other,4.279954e-07,9.999449e-01,2.293964e-34,5.047311e-17,2.639275e-05
3211,17679,vacancy in halliburton,"GB, ABE, Scotland",NaN,80000-120000,,the human resource department of halliburton o...,interested persons should possess a minimum o...,,0,...,1084,302,0,GB,other,1.481289e-02,1.000000e+00,3.829945e-01,6.291451e-12,3.139275e-04


In [241]:
train = getTrainLiklihoods(big_train)
train

/Users/MikeyMuller/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/Users/MikeyMuller/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/Users/MikeyMuller/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

/Users/MikeyMuller/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/Users/MikeyMuller/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/Users/MikeyMuller/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

/Users/MikeyMuller/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/Users/MikeyMuller/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/Users/MikeyMuller/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

/Users/MikeyMuller/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/Users/MikeyMuller/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/Users/MikeyMuller/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

/Users/MikeyMuller/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/Users/MikeyMuller/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/Users/MikeyMuller/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

...

/Users/MikeyMuller/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/Users/MikeyMuller/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/Users/MikeyMuller/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

/Users/MikeyMuller/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/Users/MikeyMuller/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/Users/MikeyMuller/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

/Users/MikeyMuller/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/Users/MikeyMuller/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/Users/MikeyMuller/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

/Users/MikeyMuller/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/Users/MikeyMuller/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/Users/MikeyMuller/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,...,description_length,requirements_length,benefits_length,country,city,likely_title_fraud,likely_profile_fraud,likely_desc_fraud,likely_req_fraud,likely_ben_fraud
3069,6817,product innovator junior,"DE, BE, Berlin",Product,NaN,babbel enables anyone to learn languages in an...,we are looking for a product innovator mf full...,requirementsyour mission is to revolutionize a...,we offer youpotential to change the world by d...,0,...,1113,593,549,DE,other,3.669685e-03,1.155213e-48,9.110289e-27,7.408142e-15,2.113146e-14
1508,16825,java tech lead,"US, CT, Hartford",NaN,NaN,esolutions inc is a talentdriven consulting co...,title java tech leadlocation hartfo...,requisition detailskey responsibilities• ...,,0,...,81,1588,0,US,other,6.502364e-03,4.293894e-10,2.223052e-13,2.646445e-22,3.193384e-04
2629,7530,sheffield estate agents office assistant appre...,"GB, , Sheffield",NaN,NaN,established on the principles that full time e...,under the national apprenticeship scheme you m...,1618 year olds only due to government fundingf...,career prospects,0,...,608,67,16,GB,other,6.925558e-09,1.034302e-84,1.914498e-21,2.862180e-10,1.140418e-05
2825,12534,it trainee,"GR, I, Athens",Internships,NaN,urlc379aa631173ed5b7c345ab3f500a9a053e509138ca...,who are wetravelplanet24 is one of the leading...,what are we looking forenergetic amp talented ...,why travelplanet24 you will acquire know...,0,...,248,1022,404,GR,athens,1.046579e-03,4.100195e-08,1.629533e-19,9.136096e-17,8.721847e-12
1427,4278,graphic artist east asia pacific division office,"US, DC, Washington",NaN,NaN,applied memetics llc is a professional service...,the graphic artist shall properly complete all...,the graphic artist shall be skilled in current...,,0,...,437,227,0,US,other,5.866722e-02,1.288956e-89,1.058773e-13,8.323793e-10,3.193384e-04
496,8311,english teacher overseas conversational,"US, TX, Dallas",NaN,NaN,we help teachers get safe amp secure jobs abroad,jobs in china amp korea1400 usd monthly 200 c...,university degree tefl tesol celta or teachi...,see job description,0,...,176,126,19,US,other,7.950178e-07,9.999523e-01,6.322053e-22,4.939436e-16,2.430942e-05
2409,7755,support worker personal assistant,"GB, EDH, Edinburgh South and South East",NaN,NaN,social care alba is the highest graded care at...,social care alba is the highest quality care a...,key accountabilities would beto provide indivi...,this is your chance to become part of our cont...,0,...,1063,1164,790,GB,other,1.257950e-02,2.107262e-42,3.161614e-21,1.730560e-29,1.063034e-16
3456,14001,process engineer mechanical,"US, CA, Bakersfield",Oil & Energy,NaN,...,we are a fullservice engineering project manag...,experience preferredpe registration a plus pre...,,0,...,1197,227,0,US,other,3.487793e-01,1.000000e+00,1.000000e+00,9.999992e-01,2.834966e-04
3026,13645,executive assistant,"US, FL, Boca Raton",NaN,35000-55000,marc bell capital partners is family home offi...,job description the ideal candidate will be ex...,must have excellent verbal and written skills ...,2014 employee benefits summaryhealthcare cover...,0,...,1454,187,3932,US,other,2.537656e-02,3.692152e-13,1.424072e-10,1.227074e-05,2.969866e-05
2481,708,bdc agent,"US, TX, Dickinson",BDC,0-0,professional success is a rapidly flourishing ...,as one of our live chat representatives you wi...,45 plus words per minute computer proficiency...,,0,...,216,373,0,US,other,6.554017e-02,5.725520e-12,5.304683e-15,3.920870e-12,2.834966e-04


In [236]:
test.to_csv("test_data_no_nan.csv")

In [242]:
train.to_csv("train_data_no_nan.csv")